In [3]:
using Turing, Distributions, Random, MCMCChains, Plots, StatsPlots, Measures
# Set a random seed for reproducibility
Random.seed!(1234)

TaskLocalRNG()

In [51]:
# Define the Turing model
@model function dirichlet_multinomial_model(K, I, n, y)
    # Priors
    beta ~ Dirichlet([0.5,0.5])  # Prior for the base distribution (50% black, 50% white)
    alpha ~ Exponential(0.5)     # Global concentration parameter

    # Container-specific color distributions
    theta = Vector{Real}(undef, I)
    for i in 1:I
        concentration_param = alpha * beta .+ 1e-6
        theta[i] ~ Dirichlet(concentration_param)      # Sample θ_i ∼ Dirichlet(α * β)
        y[i, :] ~ Multinomial(n[i], theta[i])          # Observed counts follow a multinomial distribution
    end
end

dirichlet_multinomial_model (generic function with 2 methods)

In [29]:
# Example data
K = 2  # Number of colors (e.g., black and white)
I = 12  # Number of containers (including Test1 as I-1 and Test2 as I)
n = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 20, 0]  # Initial counts for each container, with 0 for Test2
y_mixed = [
    [5, 5],   # Observed counts for container 1
    [5, 5], 
    [5, 5],
    [5, 5],   
    [5, 5],
    [5, 5],
    [5, 5], 
    [5, 5],
    [5, 5],   
    [5, 5],
    [20, 0],    # Test1: the container we incrementally add black marbles to
    [0, 0]     # Test2: the final container with no observations
]
y_uniform = [
    [10, 0],   # Observed counts for container 1
    [0, 10], 
    [10, 0],
    [0, 10],   
    [10, 0], 
    [0, 10], 
    [10, 0],
    [0, 10],   
    [10, 0], 
    [0, 10],
    [20, 0],    # Test1: the container we incrementally add black marbles to
    [0, 0]     # Test2: the final container with no observations
]

12-element Vector{Vector{Int64}}:
 [10, 0]
 [0, 10]
 [10, 0]
 [0, 10]
 [10, 0]
 [0, 10]
 [10, 0]
 [0, 10]
 [10, 0]
 [0, 10]
 [20, 0]
 [0, 0]

In [49]:
#Fit model to mixed group observations
model = dirichlet_multinomial_model(K, I, n, y_mixed)
chain = sample(model, NUTS(), 1000, progress = false)

┌ Info: Found initial step size
└   ϵ = 0.8


Chains MCMC chain (1000×39×1 Array{Float64, 3}):

Iterations        = 501:1:1500
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 0.4 seconds
Compute duration  = 0.4 seconds
parameters        = beta[1], beta[2], alpha, theta[1][1], theta[1][2], theta[2][1], theta[2][2], theta[3][1], theta[3][2], theta[4][1], theta[4][2], theta[5][1], theta[5][2], theta[6][1], theta[6][2], theta[7][1], theta[7][2], theta[8][1], theta[8][2], theta[9][1], theta[9][2], theta[10][1], theta[10][2], theta[11][1], theta[11][2], theta[12][1], theta[12][2]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
    parameters      mean       std      mcse    ess_bulk   ess_tail      rhat  ⋯
        Symbol   Float64   Float64   Float64     Float64    Float64   Float64  ⋯

       beta[1]    0.5562    0.0814    0.0023   1237.5213   